# NebulaGraph-PyG Remote Backend 使用与代码自查

本 notebook 旨在：
- 指导用户如何将 NebulaGraph 作为 PyG Remote Backend 无缝接入 GNN 全流程
- 自用回顾并优化关键实现，确保工程质量
- 为社区 code review 提供 checklist

补充一句：WSL的IO真的垃圾！前后断断续续写了五个小时的文档能不见！现在有一种充分的死人感（指无力）。

## 目录

## 环境与依赖

In [ ]:
% git clone XXX
% pip install nebula-pyg

### NebulaGraph
NebulaGraph 是一款开源的、分布式的、易扩展的原生图数据库，也是这个项目背后的数据库。nebula-pyg 本质上是 NebulaGraph 与 PyG 之间的桥梁，方便用户在进行图神经网络（GNN）相关工作的时候，能够直接从NebulaGraph中获得数据，无需关心复杂的数据转换细节。

#### 简单了解
在正式使用 NebulaGraph 和 nebula-pyg 之前，希望您能够简单了解一下 NebulaGraph，以便你对本项目有更好的理解。
重点推荐阅读 [NebulaGraph 架构](https://docs.nebula-graph.io/3.8.0/1.introduction/3.nebula-graph-architecture/1.architecture-overview/)。NebulaGraph 由三种服务构成：Graph 服务、Meta 服务和 Storage 服务，是一种存储与计算分离的架构。[Graph](https://docs.nebula-graph.io/3.8.0/1.introduction/3.nebula-graph-architecture/3.graph-service/) 服务主要负责处理查询请求;[Meta 服务](https://docs.nebula-graph.io/3.8.0/1.introduction/3.nebula-graph-architecture/2.meta-service/)负责管理集群元数据和服务信息;[Storage 服务](https://docs.nebula-graph.io/3.8.0/1.introduction/3.nebula-graph-architecture/4.storage-service/)负责数据的存储和管理。为了实现对 Storage 服务中数据的高效并行读取，nebula-pyg 直接连接 GraphStorageClient（即Storage服务），调用其 Scan 接口。
Tips：当前的NebulaGraph社区版本并不从外部网络直接连接 GraphStorageClient，所以要确保Python环境与NebulaGraph部署在统一网络中。

#### 安装部署
安装部署的方式有很多，包括[RPM/DEB 包](https://docs.nebula-graph.com.cn/3.8.0/4.deployment-and-installation/2.compile-and-install-nebula-graph/2.install-nebula-graph-by-rpm-or-deb/),使用[Tar包](https://docs.nebula-graph.com.cn/3.8.0/4.deployment-and-installation/2.compile-and-install-nebula-graph/4.install-nebula-graph-from-tar/)，以及[Docker Compose](https://docs.nebula-graph.com.cn/3.8.0/4.deployment-and-installation/2.compile-and-install-nebula-graph/3.deploy-nebula-graph-with-docker-compose/)等等。官网的内容已经相当详细，此处不再赘述。
Tips：注意一下，目前社区版仅支持在 Linux 系统下安装 NebulaGraph，且仅支持 CentOS 7.x、CentOS 8.x、Ubuntu 16.04、Ubuntu 18.04、Ubuntu 20.04 操作系统。（本人有尝试通过tar包在Ubuntu24.04上进行安装，但稳定性未经过验证）

### Python 环境
推荐 >=3.9,<=3.12,以满足nebula-python,PyG等依赖要求
### 更快速的环境配置(conda or pip)
对于[torch](https://pytorch.org/get-started/locally/)和[PyG](https://pytorch-geometric.readthedocs.io/en/latest/install/installation.html)，推荐到各自的官方网站，找到符合自己的版本进行下载
#### Conda

In [ ]:
% conda activate your_environment_name
% conda install --yes --file requirements.txt

#### pip

In [ ]:
% pip install -r requirements.txt

### 更推荐环境配置(基于PDM)
本项目为了更好接入 [nebula-python](https://github.com/vesoft-inc/nebula-python)
在包管理器上，首先选择用[PDM](https://pdm-project.org/en/latest/),具体的安装步骤可以见[PDM Introduction Installation](https://pdm-project.org/en/latest/#installation)，以下同PDM文档中的[推荐安装方式](https://pdm-project.org/en/latest/#recommended-installation-method)

+ Linux/Mac

In [ ]:
% curl -sSL https://pdm-project.org/install-pdm.py | python3 -

+ Windows

In [ ]:
% powershell -ExecutionPolicy ByPass -c "irm https://pdm-project.org/install-pdm.py | py -"

或者采用pipx，pip等方式
+ 使用pipx

In [ ]:
% pipx install pdm

接着，使用如下命令即可自动安装 pyproject.toml 中指定的依赖

In [ ]:
% pdm install

但注意，对于 torch-geometric, torch推荐到各自的官网上，找到自己对应的版本进行安装，pyproject.toml中已经具体指定，并不适用于所有平台,可自行在 pyproject.toml 进行版本的调整。
Tips: torch, PyG 等包并没有直接提供pdm的安装方式，可以采用[PDM Configure the Project Specify index for individual packages](https://pdm-project.org/en/latest/usage/config/#specify-index-for-individual-packages) 即指定平台进行安装，也可采用下面Tips中的，直接使用 .whl方法进行安装。在pyptoject.toml中加入config，类似如下：
``` toml
[[tool.pdm.source]]
name = "private-pypi"
url = "https://download.pytorch.org/whl/cu121"
include_packages = ["torch", "torchvision", "torchaudio"]
```

Tips: 如果 pyg-lib, torch_scatter, torch_sparse 这几个包出现报错，可以到具体版本页面中，如 https://data.pyg.org/whl/torch-2.7.0+cpu.html 找到符合自己版本的 .whl, 用如下命令进行安装

In [ ]:
% pdm add "pyg_lib-0.4.0+pt27cpu-cp312-cp312-linux_x86_64.whl"

如需运行代码，按照如下命令即可

In [ ]:
% pdm run python examples/XXX.py

## 如何调用
首先恭喜你完成了环境的搭建，这意味着你已经完成了整体任务的70%。
### 数据准备
使用nebula-pyg之前，请确保NebulaGraph 中已有数据。对于数据的导入和管理，强烈推荐使用[NebulaGraph Studio](https://docs.nebula-graph.com.cn/3.8.0/nebula-studio/about-studio/st-ug-what-is-graph-studio/)该工具通过可视化界面，能更方便地帮助你完成相关操作。
Tips：你的数据一定要满足节点存在可数或可计算的特征。
[ ] 准备一个数据集

### 环境准备
这里的环境准备是指如何连接gclient以及sclient。
其中，USER可以默认使用root，PASSWORD则为nebula。
可以观察到，因为处于统一网络，所以可以使用graphd，metad0等等进行连接

In [ ]:
from nebula3.Config import Config
from nebula3.gclient.net import ConnectionPool
from nebula3.sclient.GraphStorageClient import GraphStorageClient
from nebula3.mclient import MetaCache


config = Config()
connection_pool = ConnectionPool()
connection_pool.init([("graphd", 9669)], config)
# connection_pool.init([("host.docker.internal", 9669)], config)
gclient = connection_pool.get_session(USER, PASSWORD)

meta_cache = MetaCache([("metad0", 9559), ("metad1", 9559), ("metad2", 9559)], 50000)
sclient = GraphStorageClient(meta_cache)

### VID
可以简单了解一下[VID](https://docs.nebula-graph.com.cn/3.8.0/1.introduction/3.vid/),VID 是图空间中用于唯一标识一个节点的 ID，NebulaGraph 支持 INT64 和 FIXED_STRING(<N>) 两种类型。
在PyG中，[PyG对edge_index有\[0,num_nodes-1\]的要求](https://pytorch-geometric.readthedocs.io/en/latest/get_started/introduction.html#:~:text=Note%20that%20it,running%20validate()%3A)。
因此 VID 需要进行映射处理。
在设计映射方案时，曾考虑过 KV-Cache，经过 Wey-gu 老师建议，结合两个关键点：1. 考不考虑动态场景 2.删除tag的时候连续序号需要怎么处理。最后采用了Snapshot这个方案，并使用 Python 原生的 pickle 进行持久化。
因此，使用前需要先执行生成 Snapshot 的操作：

In [ ]:
% python3 gen_snapshot.py

执行后，会生成一个 pickle 文件，包含以下内容：
+ vid_to_idx 和 idx_to_vid 的双向映射
+ vid_to_tag（节点分类信息）
+ edge_type_groups 三元组信息

之后，即可在代码中，直接调用 .pkl 文件

In [ ]:
import pickle

with open(SNAPSHOT_PATH, "rb") as f:
    snapshot = pickle.load(f)

### NebulaPyG
现在，你只需要创建一个NebulaPyG对象，调用get_torch_geometric_remote_backend()方法，所有的处理细节都已经封装好了

In [ ]:
from nebula_pyg.nebula_pyg import NebulaPyG

nebula_pyg = NebulaPyG(gclient, sclient, space, snapshot)
feature_store, graph_store = nebula_pyg.get_torch_geometric_remote_backend()

### NeighborLoader
现在，你只需要从PyG中调用NeighborLoader，就可将数据进行载入

In [ ]:
from torch_geometric.loader import NeighborLoader

input_nodes = list(range(len(snapshot['idx_to_vid']['player'])))

loader = NeighborLoader(
    data=(feature_store, graph_store),
    num_neighbors={
        ('player', 'follow', 'player'): [10, 10],
        ('player', 'serve', 'team'): [10, 10],
    },
    batch_size=32,
    input_nodes=('player', input_nodes),
    num_workers=0,
    filter_per_worker=True,
)

### 训练
最后，你就可以调用你的模型，进行训练啦！

In [ ]:
for batch in loader:
    # batch['player'].x, batch['player', 'follow', 'player'].edge_index ...
    print("batch:", batch)
    # 可以直接放进你的 GNN forward 函数